# CountMatch Matcher Development

This notebook is a testbed for developing CountMatch's matcher.

In [4]:
%matplotlib inline
import sys
sys.path.append('../')
import importlib
import matplotlib.pyplot as plt
import numpy as np
import knowyourdata as kyd

import pandas as pd
from traffic_prophet import cfg
import pathlib, os
import configparser

from traffic_prophet import connection
from traffic_prophet.countmatch import reader
from traffic_prophet.countmatch import growthfactor as gf
from traffic_prophet.countmatch import neighbour

defaultcolours = plt.rcParams['axes.prop_cycle'].by_key()['color']

filepath = pathlib.Path.home().joinpath('.charlesconfig')
if os.path.isfile(filepath):
    vol_conn = connection.Connection(filepath, 'POSTGRES',
                                     'czhu.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'POSTGRES',
                                    'czhu.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['MAPBOX']['token']
else:
    filepath = pathlib.Path.home().joinpath('cf.txt')
    vol_conn = connection.Connection(filepath, 'localpg',
                                     'prj_vol.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'localpg',
                                    'gis.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['mapbox']['token']

In [5]:
rdr = reader.Reader(vol_conn)
%time rdr.read()

OperationalError: could not connect to server: Connection refused
	Is the server running on host "10.160.12.47" and accepting
	TCP/IP connections on port 5432?


In [3]:
gf.get_growth_factors(rdr)

AttributeError: 'NoneType' object has no attribute 'keys'

In [ ]:
ptc_ids = np.unique(np.abs(list(rdr.ptcs.keys())))
nb = neighbour.NeighbourLonLatEuclidean(ll_conn, 5, ptc_ids)
%time nb.find_neighbours()

## Growth Factor Diagnostic

In [ ]:
growth_factors = pd.DataFrame([(x.count_id, x.growth_factor) for x in rdr.ptcs.values()])
growth_factors.columns = ['count_id', 'growth_factor']
growth_factors['n_years'] = [x.data['AADT'].shape[0] for x in rdr.ptcs.values()]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
growth_factors['growth_factor'].plot(kind='hist', bins=50, ax=ax)
ax.set_xlabel("Year-on-year growth factor");

In [ ]:
growth_factors['deviation'] = np.abs(growth_factors['growth_factor'].values - 1.)

In [ ]:
growth_factors.sort_values('deviation', ascending=False).head(20)

In [ ]:
rdr.ptcs[-1147347].data['MADT']

In [ ]:
rdr.ptcs[20044187].data['MADT']

In [ ]:
rdr.ptcs[20044187].data['Daily Count'].loc[(2016, 130):(2016, 160), :]

In [ ]:
rdr.ptcs[20044187].data['Daily Count'].loc[(2017, 130):(2017, 160), :]

## How Close are PTCs?

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
nb.data['Distances'].apply(lambda x: x[0]).plot(kind='hist', bins=50, ax=ax)
ax.set_xlabel("Distance from short term count to nearest permanent count (km)");

In [ ]:
kyd.kyd(nb.data['Distances'].apply(lambda x: x[0]));

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
nb.data['Distances'].apply(lambda x: x[1]).plot(kind='hist', bins=50, ax=ax, color=defaultcolours[1])
ax.set_xlabel("Distance from short term count to second nearest permanent count (km)");

In [ ]:
kyd.kyd(nb.data['Distances'].apply(lambda x: x[1]));

## Playground

### NaN imputation

In [ ]:
rdr.ptcs.keys()

In [ ]:
null_ks = []
for k in sorted(list(rdr.ptcs.keys())):
    if np.any(rdr.ptcs[k].data['DoMADT'].isnull()):
        null_ks.append(k)

In [ ]:
null_ks

In [ ]:
rdr.ptcs[-13503833].data['DoM Factor']

In [ ]:
rdr.ptcs[-13503833].data['Daily Count'].loc[(2010, 120):(2010, 150), :]

In [ ]:
rdr.ptcs[-13503833].data['DoM Factor'].T.plot(figsize=(12,10), ylim=(0, 1.5))

In [ ]:
# # https://scikit-learn.org/stable/auto_examples/impute/plot_iterative_imputer_variants_comparison.html#sphx-glr-auto-examples-impute-plot-iterative-imputer-variants-comparison-py
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

In [ ]:
IterativeImputer().fit_transform(rdr.ptcs[-13503833].data['DoM Factor'].values)[4, :]

In [ ]:
rdr.ptcs[-13503833].data['DoM Factor'].values[4, :]

### DoMSTTC emulator

In [ ]:
# equivalent to isite == 1, which scans through Ms_abs one by one
sttc = rdr.sttcs[-12448867]
want_year = 2019

sttc_current = sttc.data.iloc[0, :]
sttc_dow = sttc_current['Date'].dayofweek
sttc_year = sttc_current['Date'].year

In [ ]:
# temp1 and temp2, sort of
neighbours, distances = nb.get_neighbours(sttc.centreline_id)

In [ ]:
# We don't need sel_ids, since we store data for each neighbour in its own rdr object.
neighbours

In [ ]:
# GR_STTC=mean(GR(:,1));
growth_rate_citywide = np.mean([v.growth_factor for v in rdr.ptcs.values()])

In [ ]:
growth_rate_citywide

In [ ]:
# Pick a PTC
i = 2
cptc = rdr.ptcs[neighbours[i]]

In [ ]:
# Ratio between AADT and daily count.

# Equivalent to setting D_all
cptc.data['DoY Ratio'] = cptc.data['Daily Count'].loc[:, ['Date']].copy()
for year in cptc.data['Daily Count'].index.levels[0].values:
    cptc.data['DoY Ratio'].loc[year, 'Day-to-AADT Ratio'] = (cptc.data['AADT'].at[year, 'AADT'] /
                                                             cptc.data['Daily Count'].loc[year, 'Daily Count']).values

In [ ]:
cptc.data['DoY Ratio'].reset_index(inplace=True)
cptc.data['DoY Ratio']['Month'] = cptc.data['DoY Ratio']['Date'].dt.month
cptc.data['DoY Ratio']['Day of Week'] = cptc.data['DoY Ratio']['Date'].dt.dayofweek

In [ ]:
cptc.data['DoY Ratio']['Day of Week'].unique()

In [ ]:
# If day-of-week exists in cptc, take average for the MADT, DoM Factor, DoMADT and Day-to-AADT ratio,
# weighted by the number of days of week in each month that are the STTC day-of-week, for the closest
# year with the day-of-week available.
cptc_doy_dow = cptc.data['DoY Ratio'].loc[cptc.data['DoY Ratio']['Day of Week'] == sttc_dow, :]

# Number of days of the week in each year and month.
N_days = (cptc.data['DoY Ratio'].reset_index().groupby(['Year', 'Month', 'Day of Week'])['Day-to-AADT Ratio']
          .count().unstack())

if cptc_doy_dow.shape[0]:
    # If there's a tie for the closest year, this takes the closest year BEFORE the STTC
    # year (since unique_years is ordered and np.argmin returns the first smallest value).
    unique_years = cptc_doy_dow['Year'].unique()
    closest_year = unique_years[np.argmin(np.abs(unique_years - sttc_year))]
    
    day_to_aadt_ratio_avg = (cptc_doy_dow
                             .loc[cptc_doy_dow['Year'] == closest_year, 'Day-to-AADT Ratio']
                             .mean(skipna=True))

    # Double loc is shortest way I've discovered to get single-index series.
    weights = N_days.loc[closest_year, :].loc[:, sttc_dow]
    madt_avg = np.average(cptc.data['MADT'].loc[closest_year, 'MADT'][weights.index].values, weights=weights.values)
    dom_avg = np.average(cptc.data['DoM Factor'].loc[closest_year, :].loc[weights.index, sttc_dow], weights=weights.values)
    domadt_avg = np.average(cptc.data['DoMADT'].loc[closest_year, :].loc[weights.index, sttc_dow], weights=weights.values)

    aadt_closest_year = cptc.data['AADT'].at[closest_year, 'AADT']
# If the day of week doesn't exist, just take the averages for the closest year, weighted by all
# the values.
else:
    unique_years = cptc.data['Year'].unique()
    closest_year = unique_years[np.argmin(np.abs(unique_years - sttc_year))]
    day_to_aadt_ratio_avg = (cptc.data['DoY Ratio']
                             .loc[cptc.data['Year'] == closest_year, 'Day-to-AADT Ratio']
                             .mean(skipna=True))

    n_days_closest_year = N_days.loc[closest_year, :]

    madt_weights = n_days_closest_year.sum(axis=1, skipna=True)
    madt_avg = np.average(cptc.data['MADT'].loc[closest_year, 'MADT'][madt_weights.index].values, weights=madt_weights.values)

    dom_closest_year = cptc.data['DoM Factor'].loc[closest_year, :]
    assert np.array_equal(dom_closest_year.index, n_days_closest_year.index)
    assert np.array_equal(dom_closest_year.columns, n_days_closest_year.columns)
    dom_avg = np.average(dom_closest_year.values, weights=n_days_closest_year.values)
    domadt_avg = np.average(cptc.data['DoMADT'].loc[closest_year, :].values, weights=n_days_closest_year.values)
    
    assert np.array_equal(dom_closest_year.index, n_days_closest_year.index)

In [ ]:
# Mothballed algos
# Get unique year for each day of week in DoY Ratio
# def closest_year(x, ref_year):
#     unique_years = x.unique()
#     return unique_years[np.argmin(np.abs(unique_years - ref_year))]

# cptc.data['DoY Ratio'].groupby('Day of Week')['Year'].apply(closest_year, ref_year=2010)

# Find all months in the closest year that have data.
# months_to_include = (cptc.data['DoM Factor']
#                      .loc[closest_year:closest_year, sttc_current['Date'].dayofweek]
#                      .dropna()).index.levels[1].values

In [ ]:
print(closest_year, day_to_aadt_ratio_avg, madt_avg, dom_avg, domadt_avg, aadt_closest_year)

In [ ]:
AADT_prelim = sttc.data['Daily Count'].mean() * day_to_aadt_ratio_avg * growth_rate_citywide**(want_year - sttc_year)
MADT_pj = sttc_current['Daily Count'] * dom_avg * growth_rate_citywide**(want_year - sttc_year)
MF_STTC = MADT_pj / AADT_prelim
MF_PTC = madt_avg / aadt_closest_year

In [ ]:
print(AADT_prelim, MADT_pj, MF_STTC, MF_PTC)

We can simplify this significantly by pre-calculating all these values.

In [ ]:
p = cptc

available_years = p.data['Daily Count'].index.levels[0].values

# Get ratio between AADT and daily count
doyr = p.data['Daily Count'].loc[:, ['Date']].copy()
for year in available_years:
    doyr.loc[year, 'Day-to-AADT Ratio'] = (
        p.data['AADT'].at[year, 'AADT'] /
        p.data['Daily Count'].loc[year, 'Daily Count']).values
doyr.reset_index(inplace=True)
doyr['Month'] = doyr['Date'].dt.month
doyr['Day of Week'] = doyr['Date'].dt.dayofweek

# Get day-to-AADT ratios for each day of week and year.
ptc_mse_dow = pd.DataFrame(
    doyr.groupby(['Day of Week', 'Year'])['Day-to-AADT Ratio'].mean())

# Number of days of the week in each year and month.
N_days = (doyr.reset_index()
          .groupby(['Year', 'Month', 'Day of Week'])['Day-to-AADT Ratio']
          .count().unstack())

madt_avg = []
dom_avg = []
domadt_avg = []
for dow, year in ptc_mse_dow.index:
    # Double loc is shortest way I've discovered to get single-index series.
    weights = N_days.loc[year, :].loc[:, dow]

    madt_avg.append(np.average(p.data['MADT'].loc[year, 'MADT'][weights.index].values, weights=weights.values))
    dom_avg.append(np.average(p.data['DoM Factor'].loc[year, :].loc[weights.index, dow], weights=weights.values))
    domadt_avg.append(np.average(p.data['DoMADT'].loc[year, :].loc[weights.index, dow], weights=weights.values))

ptc_mse_dow['MADT Avg.'] = madt_avg
ptc_mse_dow['DoM Factor Avg.'] = dom_avg
ptc_mse_dow['DoMADT Avg.'] = domadt_avg

ptc_mse_yo = pd.DataFrame(doyr.groupby('Year')['Day-to-AADT Ratio'].mean())

madt_avg = []
dom_avg = []
domadt_avg = []
for year in ptc_mse_yo.index:
    n_days_year = N_days.loc[year, :]
    madt_weights = n_days_year.sum(axis=1, skipna=True)

    madt_avg.append(np.average(p.data['MADT'].loc[year, 'MADT'][madt_weights.index].values, weights=madt_weights.values))

    dom_year = cptc.data['DoM Factor'].loc[year, :]
    dom_avg.append(np.average(dom_year.values, weights=n_days_year.values))
    domadt_avg.append(np.average(p.data['DoMADT'].loc[year, :].values, weights=n_days_year.values))

ptc_mse_yo['MADT Avg.'] = madt_avg
ptc_mse_yo['DoM Factor Avg.'] = dom_avg
ptc_mse_yo['DoMADT Avg.'] = domadt_avg

In [ ]:
ptc_mse_dow

In [ ]:
ptc_mse_yo

In [ ]:
if sttc_dow in ptc_mse_dow.index.levels[0]:
    unique_years = ptc_mse_dow.loc[sttc_dow].index.values
    closest_year = unique_years[
                        np.argmin(np.abs(unique_years - sttc_year))]
ptc_mse_dow.loc[(sttc_dow, closest_year)]

## Functionalize DoMSTTC emulator

In [ ]:
def nanaverage(x, axis=None, weights=None):
    if weights is None:
        return np.nanmean(x)
    notnull = ~(np.isnan(x) | np.isnan(weights))
    return np.average(x[notnull], axis=axis, weights=weights[notnull])

x = np.random.randint(0, 331, 144).astype(float).reshape(12,12)
w = np.random.rand(144).reshape(12,12)
x[7, 1] = x[9, 2] = x[3, 1] = np.nan
w[2, 2] = np.nan

xr = x.ravel()
wr = w.ravel()
rnotnull = ~(np.isnan(xr) | np.isnan(wr))
assert np.isclose(np.average(xr[rnotnull], weights=wr[rnotnull]), nanaverage(x, weights=w), atol=1e-8, rtol=1e-10)

In [ ]:
def mse_preprocess_ptc_loop(p):
    available_years = p.data['Daily Count'].index.levels[0].values

    # Get ratio between AADT and daily count
    doyr = p.data['Daily Count'].loc[:, ['Date']].copy()
    for year in available_years:
        doyr.loc[year, 'Day-to-AADT Ratio'] = (
            p.data['AADT'].at[year, 'AADT'] /
            p.data['Daily Count'].loc[year, 'Daily Count']).values
    doyr.reset_index(inplace=True)
    doyr['Month'] = doyr['Date'].dt.month
    doyr['Day of Week'] = doyr['Date'].dt.dayofweek
    
    # Number of days of the week in each year and month.  Fill NaNs with
    # 0.
    N_days = (doyr.reset_index()
              .groupby(['Year', 'Month', 'Day of Week'])['Day-to-AADT Ratio']
              .count().unstack(fill_value=0.))

    # Create two arrays - first breaks down values by day-of-week and year.
    # First, get day-to-AADT ratios for each day of week and year.
    ptc_mse_ydow = pd.DataFrame(
        doyr.groupby(['Day of Week', 'Year'])['Day-to-AADT Ratio'].mean())

    # Then, for each day-of-week, calculate MADT, DoMADT, etc. averages
    # weighted by number of days-of-the-week available in the month.
    madt_avg = []
    dom_avg = []
    domadt_avg = []
    for dow, year in ptc_mse_ydow.index:
        # Double loc is shortest way I've discovered to get single-index
        # series.
        weights = N_days.loc[year, :].loc[:, dow]

        madt_avg.append(np.average(
            p.data['MADT'].loc[year, 'MADT'][weights.index].values,
            weights=weights.values))
        dom_avg.append(nanaverage(
            p.data['DoM Factor'].loc[year, :].loc[weights.index, dow],
            weights=weights.values))
        domadt_avg.append(nanaverage(
            p.data['DoMADT'].loc[year, :].loc[weights.index, dow],
            weights=weights.values))
        
    ptc_mse_ydow['MADT Avg.'] = madt_avg
    ptc_mse_ydow['DoM Factor Avg.'] = dom_avg
    ptc_mse_ydow['DoMADT Avg.'] = domadt_avg
    
    if (ptc_mse_ydow['MADT Avg.'].isnull().values.any() or
            ptc_mse_ydow['DoM Factor Avg.'].isnull().values.any() or
            ptc_mse_ydow['DoM Factor Avg.'].isnull().values.any()):
        raise ValueError("yeah, you shouldn't be getting nulls for ydow tables.")

    # Now, create a second array that breaks down the same values but only
    # by year.
    ptc_mse_yo = pd.DataFrame(doyr.groupby('Year')['Day-to-AADT Ratio'].mean())

    madt_avg = []
    dom_avg = []
    domadt_avg = []
    for year in ptc_mse_yo.index:
        n_days_year = N_days.loc[year, :]
        madt_weights = n_days_year.sum(axis=1, skipna=True)

        madt_avg.append(np.average(
            p.data['MADT'].loc[year, 'MADT'][madt_weights.index].values,
            weights=madt_weights.values))

        dom_year = p.data['DoM Factor'].loc[year, :]
        dom_avg.append(nanaverage(
            dom_year.values, weights=n_days_year.values))
        domadt_avg.append(nanaverage(
            p.data['DoMADT'].loc[year, :].values, weights=n_days_year.values))

    ptc_mse_yo['MADT Avg.'] = madt_avg
    ptc_mse_yo['DoM Factor Avg.'] = dom_avg
    ptc_mse_yo['DoMADT Avg.'] = domadt_avg

    if (ptc_mse_yo['MADT Avg.'].isnull().values.any() or
            ptc_mse_yo['DoM Factor Avg.'].isnull().values.any() or
            ptc_mse_yo['DoM Factor Avg.'].isnull().values.any()):
        raise ValueError("yeah, you shouldn't be getting nulls for yo tables.")

    return doyr, ptc_mse_ydow, ptc_mse_yo

def mse_preprocess_ptcs(ptcs):
    for p in ptcs.values():
        doyr, ptc_mse_ydow, ptc_mse_yo = mse_preprocess_ptc_loop(p)
        p.mse_pp = {
            'Day-to-AADT': doyr,
            'MSE_yDoW': ptc_mse_ydow,
            'MSE_y': ptc_mse_yo
        }

In [ ]:
%time mse_preprocess_ptcs(rdr.ptcs)

In [ ]:
rdr.ptcs[-14659244].mse_pp['MSE_yDoW']['MADT Avg.']

In [ ]:
rdr.ptcs[8540609].mse_pp

In [ ]:
tc_dc = rdr.sttcs[-175].data.reset_index()
tc_dc['Day of Week'] = tc_dc['Date'].dt.dayofweek

for i, row in tc_dc.iterrows():
    ryear, rdow = row['Year'], row['Day of Week']
    print(ryear, rdow, rdow in rdr.ptcs[neighbours[2]].mse_pp['MSE_yDoW'].index.levels[0])

In [ ]:
rdr.ptcs[neighbours[2]].mse_pp['MSE_yDoW'].loc[(0, 2015)]

In [ ]:
day_to_aadt_ratio_avg, madt_avg, dom_avg, domadt_avg = rdr.ptcs[neighbours[2]].mse_pp['MSE_yDoW'].loc[(0, 2015)]

In [ ]:
print(day_to_aadt_ratio_avg, madt_avg, dom_avg, domadt_avg)

In [ ]:
rdr.ptcs[neighbours[2]].mse_pp['MSE_yDoW'].loc[0].index

In [ ]:
# equivalent to isite == 1, which scans through Ms_abs one by one
sttc = rdr.sttcs[-12448867]
want_year = 2019

sttc_current = sttc.data.iloc[0, :]
sttc_dow = sttc_current['Date'].dayofweek
sttc_year = sttc_current['Date'].year

AADT_prelim = sttc.data['Daily Count'].mean() * day_to_aadt_ratio_avg * growth_rate_citywide**(want_year - sttc_year)
MADT_pj = sttc_current['Daily Count'] * dom_avg * growth_rate_citywide**(want_year - sttc_year)
MF_STTC = MADT_pj / AADT_prelim
MF_PTC = madt_avg / aadt_closest_year

In [ ]:
rdr.ptcs[neighbours[2]].mse_pp['MSE_y'].loc[2015]

In [ ]:
tc_dc

In [ ]:
tc_dc

In [ ]:
tc = rdr.sttcs[-12448867]
rptcs = rdr.ptcs
# Find neighbouring PTCs by first finding neighbouring centreline IDs,
# then checking if either direction exists in rptcs.
neighbours, distances = nb.get_neighbours(tc.centreline_id)
neighbour_ptcs = [rptcs[n] for n in
                  [-nbrs for nbrs in neighbours] + neighbours
                  if n in rptcs.keys()]

# Declare the columns in the final saved data frame.
tc_year = []
tc_dayofyear = []
tc_ptcid = []
tc_day_to_aadt_ratio_avg = []
tc_madt_avg = []
tc_dom_avg = []
tc_domadt_avg = []
tc_closest_year = []
tc_aadt_closest_year = []

tc_dc = tc.data.reset_index()
tc_dc['Day of Week'] = tc_dc['Date'].dt.dayofweek

for i, row in tc_dc.iterrows():
    ryear, rdow = row['Year'], row['Day of Week']

    for p in neighbour_ptcs:

        if rdow in p.mse_pp['MSE_yDoW'].index.levels[0]:
            unique_years = p.mse_pp['MSE_yDoW'].loc[rdow].index.values
            closest_year = unique_years[np.argmin(
                np.abs(unique_years - ryear))]

            (day_to_aadt_ratio_avg, madt_avg, dom_avg, domadt_avg) = (
               p.mse_pp['MSE_yDoW'].loc[(rdow, closest_year)])
        else:
            # Levels contain all unique years regardless if each is
            # available for every day-of-week.
            unique_years = p.mse_pp['MSE_yDoW'].index.levels[1].values
            closest_year = unique_years[np.argmin(
                np.abs(unique_years - ryear))]

            (day_to_aadt_ratio_avg, madt_avg, dom_avg, domadt_avg) = (
               p.mse_pp['MSE_y'].loc[closest_year])

        aadt_closest_year = p.data['AADT'].at[closest_year, 'AADT']

        tc_year.append(ryear)
        tc_dayofyear.append(row['Day of Year'])
        tc_ptcid.append(p.count_id)
        tc_day_to_aadt_ratio_avg.append(day_to_aadt_ratio_avg)
        tc_madt_avg.append(madt_avg)
        tc_dom_avg.append(dom_avg)
        tc_domadt_avg.append(domadt_avg)
        tc_closest_year.append(closest_year)
        tc_aadt_closest_year.append(aadt_closest_year)

In [ ]:
tc_mse = pd.DataFrame({
    'Year': tc_year,
    'Day of Year': tc_dayofyear,
    'PTC ID': tc_ptcid,
    'PTC Day-to-AADT Ratio': tc_day_to_aadt_ratio_avg,
    'PTC MADT Avg.': tc_madt_avg,
    'PTC DoM Factor Avg.': tc_dom_avg,
    'PTC DoMADT Avg.': tc_domadt_avg,
    'PTC Closest Year AADT': tc_aadt_closest_year
})

In [ ]:
tc_mse = pd.merge(tc_dc, tc_mse, on=('Year', 'Day of Year'))

# I disagree with this, but line 95 of DoMSTTC.m seems to do it.
mean_tc_count = tc_dc['Daily Count'].mean()

tc_mse['AADT_prelim'] = (
    mean_tc_count * tc_mse['PTC Day-to-AADT Ratio'] *
    growth_rate_citywide**(want_year - tc_mse['Year']))
tc_mse['MADT_pj'] = (
    tc_mse['Daily Count'] * tc_mse['PTC DoM Factor Avg.'] *
    growth_rate_citywide**(want_year - tc_mse['Year']))
tc_mse['MF_STTC'] = tc_mse['MADT_pj'] / tc_mse['AADT_prelim']
tc_mse['MF_PTC'] = (tc_mse['PTC MADT Avg.'] /
                    tc_mse['PTC Closest Year AADT'])

tc.tc_mse = tc_mse

In [ ]:
tc_mse 

In [ ]:
def get_normalized_seasonal_patterns(tcs, rptcs, want_year):
    """Get normalized seasonal patterns for `tcs`.

    For STTCs, get best estimate normalized patterns and corresponding PTC
    normalized patterns to check for MSE (Eqn. 6 in Bagheri).  For PTCs, get
    best estimate from nearby PTCs and check as a part of validation.
    """
    growth_rate_citywide = np.mean([v.growth_factor for v in rptcs.values()])

    for tc in tcs.values():
        # Find neighbouring PTCs by first finding neighbouring centreline IDs,
        # then checking if either direction exists in rptcs.
        neighbours, distances = nb.get_neighbours(tc.centreline_id)
        neighbour_ptcs = [rptcs[n] for n in
                          [-nbrs for nbrs in neighbours] + neighbours
                          if n in rptcs.keys()]

        # Declare the columns in the final saved data frame.
        tc_year = []
        tc_dayofyear = []
        tc_ptcid = []
        tc_day_to_aadt_ratio_avg = []
        tc_madt_avg = []
        tc_dom_avg = []
        tc_domadt_avg = []
        tc_closest_year = []
        tc_aadt_closest_year = []

        if tc.is_permanent:
            tc_dc = tc.data['Daily Count'].reset_index()
        else:
            tc_dc = tc.data.reset_index()
        tc_dc['Day of Week'] = tc_dc['Date'].dt.dayofweek

        for i, row in tc_dc.iterrows():
            ryear, rdow = row['Year'], row['Day of Week']

            for p in neighbour_ptcs:

                if rdow in p.mse_pp['MSE_yDoW'].index.levels[0]:
                    unique_years = p.mse_pp['MSE_yDoW'].loc[rdow].index.values
                    closest_year = unique_years[np.argmin(
                        np.abs(unique_years - ryear))]

                    (day_to_aadt_ratio_avg, madt_avg, dom_avg, domadt_avg) = (
                       p.mse_pp['MSE_yDoW'].loc[(rdow, closest_year)])
                else:
                    # Levels contain all unique years regardless if each is
                    # available for every day-of-week.
                    unique_years = p.mse_pp['MSE_yDoW'].index.levels[1].values
                    closest_year = unique_years[np.argmin(
                        np.abs(unique_years - ryear))]

                    (day_to_aadt_ratio_avg, madt_avg, dom_avg, domadt_avg) = (
                       p.mse_pp['MSE_y'].loc[closest_year])
                
                if madt_avg is np.nan:
                    raise ValueError("ummmmm, this can't be NaN.")

                aadt_closest_year = p.data['AADT'].at[closest_year, 'AADT']

                tc_year.append(ryear)
                tc_dayofyear.append(row['Day of Year'])
                tc_ptcid.append(p.count_id)
                tc_day_to_aadt_ratio_avg.append(day_to_aadt_ratio_avg)
                tc_madt_avg.append(madt_avg)
                tc_dom_avg.append(dom_avg)
                tc_domadt_avg.append(domadt_avg)
                tc_closest_year.append(closest_year)
                tc_aadt_closest_year.append(aadt_closest_year)

        tc_mse = pd.DataFrame({
            'Year': tc_year,
            'Day of Year': tc_dayofyear,
            'PTC ID': tc_ptcid,
            'PTC Day-to-AADT Ratio': tc_day_to_aadt_ratio_avg,
            'PTC MADT Avg.': tc_madt_avg,
            'PTC DoM Factor Avg.': tc_dom_avg,
            'PTC DoMADT Avg.': tc_domadt_avg,
            'PTC Closest Year AADT': tc_aadt_closest_year
        })

        tc_mse = pd.merge(tc_dc, tc_mse, on=('Year', 'Day of Year'))

        # I disagree with this, but line 95 of DoMSTTC.m seems to do it.
        mean_tc_count = tc_dc['Daily Count'].mean()

        tc_mse['AADT_prelim'] = (
            mean_tc_count * tc_mse['PTC Day-to-AADT Ratio'] *
            growth_rate_citywide**(want_year - tc_mse['Year']))
        tc_mse['MADT_pj'] = (
            tc_mse['Daily Count'] * tc_mse['PTC DoM Factor Avg.'] *
            growth_rate_citywide**(want_year - tc_mse['Year']))
        tc_mse['MF_STTC'] = tc_mse['MADT_pj'] / tc_mse['AADT_prelim']
        tc_mse['MF_PTC'] = (tc_mse['PTC MADT Avg.'] /
                            tc_mse['PTC Closest Year AADT'])

        tc.tc_mse = tc_mse

In [ ]:
%time get_normalized_seasonal_patterns(rdr.sttcs, rdr.ptcs, 2018)

In [ ]:
%time get_normalized_seasonal_patterns(rdr.ptcs, rdr.ptcs, 2018)

In [ ]:
rdr.ptcs[8540609].tc_mse

## MSE Calculator

- Get mean squared error for each STTC/PTC pair (regardless of year)
- Get mean D_ij for each STTC/PTC pair (regardless of year)
- Then for each STTC:
  - Determine the PTC that gives the minimum MSE.  Use the D_ij from that PTC.
  - Find the closest year of counts to the want_year
  - Find the mean daily traffic, mean PTC AADT, and mean growth factor (all these means are row-by-row, and many are means of the same number since there's a lot of repetition in DoM_STTC)
  - Get the growth rate; right now it's *still* the global growth rate averaged over all PTCs.
  - Determine an AADT estimate using Mean daily traffic of closest year * Dij(minimum MSE PTC station) * GR ^ (want_year - closest_year)
- For each PTC:
  - Find the year closest to wanted year
  - Find the mean daily traffic (averaged over rows, with no heed taken to weight by monthly representation)
  - Find the mean growth factor for that station
  - Determine an AADT estimate using mean daily traffic * GR^(want_year - closest_year)

In [ ]:
rdr.sttcs[117].tc_mse

In [ ]:
rdr.sttcs[117].tc_mse['Square Deviation'] = (
    rdr.sttcs[117].tc_mse['MF_STTC'] - rdr.sttcs[117].tc_mse['MF_PTC'])**2

In [ ]:
rdr.sttcs[117].tc_mse.groupby('PTC ID')[['Square Deviation', 'PTC Day-to-AADT Ratio']].mean()

In [ ]:
ptc_dijs = (rdr.sttcs[117].tc_mse
            .groupby('PTC ID')[['Square Deviation', 'PTC Day-to-AADT Ratio']]
            .mean())
id_min_mse_ptc = ptc_dijs['Square Deviation'].idxmin()
dij_min_mse_ptc = ptc_dijs.at[id_min_mse_ptc, 'PTC Day-to-AADT Ratio']

In [ ]:
print(id_min_mse_ptc, dij_min_mse_ptc)

In [ ]:
sttc_years = rdr.sttcs[117].data.index.levels[0].values
closest_year = sttc_years[np.abs(want_year - sttc_years).argmin()]

In [ ]:
growth_rate_citywide = np.mean([v.growth_factor for v in rdr.ptcs.values()])

In [ ]:
sttc_avg_daily_count_closest_year = rdr.sttcs[117].data.loc[closest_year]['Daily Count'].mean()

In [ ]:
aadt_estimate = (sttc_avg_daily_count_closest_year * dij_min_mse_ptc *
                 growth_rate_citywide**(want_year - closest_year))

In [ ]:
aadt_estimate

In [ ]:
ptc_years = rdr.ptcs[8540609].data['AADT'].index.values
closest_year = ptc_years[np.abs(want_year - ptc_years).argmin()]
ptc_avg_daily_count_cy = rdr.ptcs[8540609].data['Daily Count'].loc[closest_year]['Daily Count'].mean()
aadt_estimate = ptc_avg_daily_count_cy * rdr.ptcs[8540609].growth_factor**(want_year - closest_year)

In [ ]:
aadt_estimate

### Functionalize AADT Estimator

In [ ]:
def get_aadt_estimates_sttc(rdr, want_year):

    sttc_count_id = []
    sttc_ptc_id_minmse = []
    sttc_dij_minmse = []
    sttc_closest_year = []
    sttc_aadt_est = []
    
    # Average of all PTC growth rates.
    sttc_gr = np.mean([v.growth_factor for v in rdr.ptcs.values()])

    for tc in rdr.sttcs.values():
        # Calculate pointwise square error between STTC and neighbouring PTCs,
        # ignoring year.
        tc.tc_mse['Square Deviation'] = (
            tc.tc_mse['MF_STTC'] - tc.tc_mse['MF_PTC'])**2

        # Determine minimum MSE between STTC and each PTC.
        dijs = (tc.tc_mse
                .groupby('PTC ID')[['Square Deviation',
                                    'PTC Day-to-AADT Ratio']]
                .mean())
        ptcid_mmse = dijs['Square Deviation'].idxmin()
        dij_mmseptc = dijs.at[ptcid_mmse, 'PTC Day-to-AADT Ratio']
        
        # Determine average daily count for most recent year to wanted year.
        sttc_years = tc.data.index.levels[0].values
        closest_year = sttc_years[np.abs(want_year - sttc_years).argmin()]
        sttc_daily_count_cyavg = tc.data.loc[closest_year]['Daily Count'].mean()
        
        aadt_estimate = (
            sttc_daily_count_cyavg * dij_mmseptc *
            sttc_gr**(want_year - closest_year))
        
        # Add to lists.
        sttc_count_id.append(tc.count_id)
        sttc_ptc_id_minmse.append(ptcid_mmse)
        sttc_dij_minmse.append(dij_mmseptc)
        sttc_closest_year.append(closest_year)
        sttc_aadt_est.append(aadt_estimate)
        
    sttc_aadt = pd.DataFrame({
        'Count ID': sttc_count_id,
        'PTC ID': sttc_ptc_id_minmse,
        'D_ij': sttc_dij_minmse,
        'Closest Year': sttc_closest_year,
        'AADT Estimate': sttc_aadt_est})

    ptc_count_id = []
    ptc_closest_year = []
    ptc_growth_factor = []
    ptc_aadt_estimate = []
    
    for tc in rdr.ptcs.values():
        ptc_years = tc.data['AADT'].index.values
        closest_year = ptc_years[np.abs(want_year - ptc_years).argmin()]
        ptc_daily_count_cyavg = tc.data['Daily Count'].loc[closest_year]['Daily Count'].mean()
        aadt_estimate = ptc_daily_count_cyavg * tc.growth_factor**(want_year - closest_year)
        
        ptc_count_id.append(tc.count_id)
        ptc_closest_year.append(closest_year)
        ptc_growth_factor.append(tc.growth_factor)
        ptc_aadt_estimate.append(aadt_estimate)
        
    ptc_aadt = pd.DataFrame({
        'Count ID': ptc_count_id,
        'Closest Year': ptc_closest_year,
        'Growth Factor': ptc_growth_factor,
        'AADT Estimate': ptc_aadt_estimate})

    return sttc_aadt, ptc_aadt

In [ ]:
%time sttc_aadt, ptc_aadt = get_aadt_estimates_sttc(rdr, 2019)

In [ ]:
ptc_aadt

In [ ]:
sttc_aadt

## Do Some Analysis

In [ ]:
import geopandas as gpd
import matplotlib.colors as colors

sql_query = ("SELECT centreline_id, fcode_desc, geom, lon, lat,"
             "ST_SetSRID(ST_MakePoint(lon, lat), 4326) point_geom "
             "FROM {table}").format(table=ll_conn.tablename)
ctrline_geoms = gpd.read_postgis(sql_query, ll_conn.connect())

In [ ]:
sttc_aadt['Centreline ID'] = sttc_aadt['Count ID'].abs()
ptc_aadt['Centreline ID'] = ptc_aadt['Count ID'].abs()

aadt_estimates = (
    pd.concat([sttc_aadt.loc[sttc_aadt['Count ID'].apply(lambda x: x not in ptc_aadt['Count ID']),
                             ['Count ID', 'Centreline ID', 'AADT Estimate']],
    ptc_aadt.loc[:, ['Count ID', 'Centreline ID', 'AADT Estimate']]]).reset_index(drop=True))

aadt_estimates = pd.merge(
    ctrline_geoms[['centreline_id', 'geom']],
    (aadt_estimates.groupby('Centreline ID')[['AADT Estimate']]
     .mean().reset_index()), how='inner',
    left_on='centreline_id', right_on='Centreline ID')
aadt_estimates = gpd.GeoDataFrame(aadt_estimates, crs={'init': 'epsg:4326'}, geometry='geom')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
aadt_estimates['AADT Estimate'].hist(ax=ax, bins=100, log=True)

In [ ]:
fig = plt.figure(figsize=(12, 9))
fig.patch.set_facecolor('#1c1c1c')
ax = fig.add_axes([0., 0., 1., 1.])
ax.axis('off')
# Not absolute minimum, for more color dynamics.
min_aadt = aadt_estimates['AADT Estimate'].quantile(0.05)
max_aadt = aadt_estimates['AADT Estimate'].max()
aadt_estimates.to_crs(epsg=3857).plot(column='AADT Estimate', ax=ax, cmap='viridis',
                                      norm=colors.LogNorm(), vmin=min_aadt, vmax=max_aadt)